<a href="https://colab.research.google.com/github/szn5400/CO2_all/blob/main/Copy_of_Copy_of_Copy_of_PINN_trial_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for mtt (MLP)  

#MTT Paper reimplementation with all data taken for rescaling rather than just training data

import sys
sys.path.append('/content/drive/MyDrive/code/MLP')

from read_data_unscaled import read
import os
import numpy as np
import glob
import collections
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from read_model import MLP_multiple
from data_to_numpy_unscaled import numpy_multi


#read data
all_pressures,all_saturations,all_permeabilities,all_porosities,all_surf_inj_rate_series,all_surf_prod_rate_series,Ks,Rs = read()

#convert to numpy
features1_tr,features2_tr,target1_tr,target2_tr,target3_tr,features1_te,features2_te,target1_te,target2_te,target3_te,min_target1,max_target1,min_target2,max_target2,min_target3,max_target3 = numpy_multi(all_pressures,all_saturations,all_permeabilities,all_porosities,all_surf_inj_rate_series,all_surf_prod_rate_series,Ks,Rs)


All files: ['/content/drive/MyDrive/data/k1r1-h.out', '/content/drive/MyDrive/data/k1r2-h.out', '/content/drive/MyDrive/data/k1r3-h.out', '/content/drive/MyDrive/data/k1r4-h.out', '/content/drive/MyDrive/data/k1r5-h.out', '/content/drive/MyDrive/data/k1r6-h.out', '/content/drive/MyDrive/data/k1r7-h.out', '/content/drive/MyDrive/data/k1r8-h.out', '/content/drive/MyDrive/data/k1r9-h.out', '/content/drive/MyDrive/data/k2r1-h.out', '/content/drive/MyDrive/data/k2r2-h.out', '/content/drive/MyDrive/data/k2r3-h.out', '/content/drive/MyDrive/data/k2r4-h.out', '/content/drive/MyDrive/data/k2r5-h.out', '/content/drive/MyDrive/data/k2r6-h.out', '/content/drive/MyDrive/data/k2r7-h.out', '/content/drive/MyDrive/data/k2r8-h.out', '/content/drive/MyDrive/data/k2r9-h.out', '/content/drive/MyDrive/data/k3r1-h.out', '/content/drive/MyDrive/data/k3r2-h.out', '/content/drive/MyDrive/data/k3r3-h.out', '/content/drive/MyDrive/data/k3r4-h.out', '/content/drive/MyDrive/data/k3r5-h.out', '/content/drive/MyDriv

In [2]:
print(len(target3_tr))
target_tr = [[0 for j in range(3)] for i in range(len(target1_tr))]
for i in range(len(target1_tr)):
  target_tr[i][0] = target1_tr[i]
  target_tr[i][1] = target2_tr[i]
  target_tr[i][2] = target3_tr[i]

3195000


In [3]:
target_tr = np.asarray(target_tr)
target_tr = target_tr.reshape(3195000,3)
print('X_train: ' + str(features1_tr.shape))
print('Y_train: ' + str(target_tr.shape))

X_train: (3195000, 7)
Y_train: (3195000, 3)


In [4]:
size_input = 7
size_hidden1 = 32
size_hidden2 = 16
size_hidden3 = 8
size_output = 3
number_of_train_examples = 50000
number_of_test_examples = 1000

In [5]:
X_train = features1_tr[:number_of_train_examples,:]
y_train = target_tr[:number_of_train_examples,:]
X_test = features1_tr[number_of_train_examples:number_of_train_examples+number_of_test_examples,:]
y_test = target_tr[number_of_train_examples:number_of_train_examples+number_of_test_examples,:]

In [6]:
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test: ' + str(X_test.shape))
print('Y_test: ' + str(y_test.shape))

X_train: (50000, 7)
Y_train: (50000, 3)
X_test: (1000, 7)
Y_test: (1000, 3)


In [7]:
import os
import numpy as np
import time
import tensorflow as tf

np.random.seed(43)
tf.random.set_seed(43)
# Split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128)
#test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)

In [8]:
class MLP(object):
  def __init__(self, size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden1: int, size of hidden layer 1
    size_hidden2: int, size of hodden layer 2
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input,self.size_hidden1,self.size_hidden2,self.size_hidden3,self.size_output, self.device =\
    size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device
    
    # Initialize weights between input layer and hidden layer1
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1]))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden1]))
    # Initialize weights between input layer and hidden layer2
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2]))
    # Initialize biases for hidden layer
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden2]))
    # Initialize weights between input layer and hidden layer3
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_hidden3]))
    # Initialize biases for hidden layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_hidden3]))
     # Initialize weights between hidden layer and output layer
    self.W4 = tf.Variable(tf.random.normal([self.size_hidden3, self.size_output]))
    # Initialize biases for output layer
    self.b4 = tf.Variable(tf.random.normal([1, self.size_output]))
    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.W4,self.b1, self.b2, self.b3, self.b4]
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
  
  def loss(self, X, y_pred, y_true, grad_flag,counter):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, self.size_output)), dtype=tf.float32)

    #loss_val = tf.cast(tf.losses.mean_squared_error(y_true_tf, y_pred_tf),dtype= tf.float64)

    return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)

  def accuracy(self, y_pred, y_true):
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, self.size_output)), dtype=tf.float32)
    correct_pred = tf.reduce_sum(tf.abs(y_true_tf-y_pred_tf))  
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    return accuracy
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(inputs,predicted, y_train, True,1)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))
        
        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    
    # Compute values in hidden layer1
    what1 = tf.matmul(X_tf, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)
    #hhat1_1 = tf.nn.dropout(hhat1, 0.25)
    
    # Compute values in hidden layer2
    what2 = tf.matmul(hhat1, self.W2) + self.b2
    hhat2 = tf.nn.relu(what2)
    #hhat2_1 = tf.nn.dropout(hhat2, 0.25)

    # Compute values in hidden layer3
    what3 = tf.matmul(hhat2, self.W3) + self.b3
    hhat3 = tf.nn.relu(what3)

    # Compute output
    output = tf.matmul(hhat3, self.W4) + self.b4
    output = tf.nn.relu(output)
    return output

In [9]:
# Set number of epochs
NUM_EPOCHS = 20

In [10]:
# Initialize model using CPU
mlp_on_cpu = MLP(size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device='gpu')

# Array to store accuracy and loss
loss_with_epoch = []
acc_with_epoch = []

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  ac = 0
  count = 0
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(1234)).batch(1)
  counter = 0
  for inputs, outputs in train_ds:
    grad_flag = True
    preds = mlp_on_cpu.forward(inputs)
    loss_total = loss_total + mlp_on_cpu.loss(inputs, preds, outputs, grad_flag, counter)
    #grad_flag = False
    #lt = lt + mlp_on_cpu.loss(inputs, preds, outputs, grad_flag, 1)
    mlp_on_cpu.backward(inputs, outputs)
    ac = ac+mlp_on_cpu.accuracy(preds, outputs)
    #ac = mlp_on_cpu.accuracy(preds, outputs)
    count += 1
  print('Number of Epoch = {} - Average celoss:= {}- Acc:= {} '.format(epoch + 1, np.sum(loss_total) / X_train.shape[0], ac/count))
  loss_with_epoch.append(np.sum(loss_total) / X_train.shape[0])
  acc_with_epoch.append(ac/count)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#For per epoch_time = Total_Time / Number_of_epochs

Number of Epoch = 1 - Average celoss:= 0.5358762890625- Acc:= 0.8991643786430359 
Number of Epoch = 2 - Average celoss:= 0.17173029296875- Acc:= 0.7461352944374084 
Number of Epoch = 3 - Average celoss:= 0.1715753515625- Acc:= 0.7454439997673035 
Number of Epoch = 4 - Average celoss:= 0.1712866015625- Acc:= 0.7443976402282715 
Number of Epoch = 5 - Average celoss:= 0.17085447265625- Acc:= 0.7428695559501648 
Number of Epoch = 6 - Average celoss:= 0.17057435546875- Acc:= 0.741967499256134 
Number of Epoch = 7 - Average celoss:= 0.170303515625- Acc:= 0.7410963773727417 
Number of Epoch = 8 - Average celoss:= 0.16995478515625- Acc:= 0.739870011806488 
Number of Epoch = 9 - Average celoss:= 0.1698241015625- Acc:= 0.7394944429397583 
Number of Epoch = 10 - Average celoss:= 0.16954853515625- Acc:= 0.7385789752006531 
Number of Epoch = 11 - Average celoss:= 0.1692567578125- Acc:= 0.7377051711082458 
Number of Epoch = 12 - Average celoss:= 0.16898865234375- Acc:= 0.7366437315940857 
Number of 